In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="Udacity-project")
exp = Experiment(workspace=ws, name="exp3")
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: Udacity-project
Azure region: eastus
Subscription id: de47103e-2da6-4f5e-88fc-d18b27fd249b
Resource group: aml-quickstarts-112571


In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
cluster_name = "comp-cluster1"

# check for existing cluster
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster:',cluster_name)
except :
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws,cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster: comp-cluster1
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


# Model hyperparameter selection using Hyperdrive

In [6]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import normal, uniform, choice
import os
import shutil

# Specify parameter sampler
ps =  RandomParameterSampling( {
        "--C": choice(1,0.01,0.001),
        "--max_iter": choice(100,200)  })

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

shutil.copy('train.py', './training')

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./training', compute_target=compute_target,entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est, hyperparameter_sampling=ps,
                                     policy= policy ,primary_metric_name='AUC',primary_metric_goal =PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=4)

In [7]:
# Submit your hyperdrive run to the experiment
hyperdrive_run = exp.submit(hyperdrive_config, show_output=True)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


In [8]:
# show the run details using widget
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [11]:
# get the best run
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('ROC AUC score:',best_run_metrics['AUC'])
print('Accuracy:', best_run_metrics['Accuracy'])
print('Regularization:',parameter_values[1])
print('Maximum Iterations:',parameter_values[3])

Best Run Id:  HD_1d8d49d0-8c4d-4ca6-94f7-6b3f7ee8ef55_2
ROC AUC score: 0.9268133721181365
Accuracy: 0.9139605462822459
Regularization: 0.01
Maximum Iterations: 200


In [17]:
import joblib
#save the model from that run.
best_run.register_model(model_path='outputs/hyperdrive_model.pkl', model_name='hyperdrive_model')

Model(workspace=Workspace.create(name='Udacity-project', subscription_id='de47103e-2da6-4f5e-88fc-d18b27fd249b', resource_group='aml-quickstarts-112571'), name=hyperdrive_model, id=hyperdrive_model:3, version=3, tags={}, properties={})

# Modelling hyperparameter selection using AutoML

In [18]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
automl_data = TabularDatasetFactory.from_delimited_files(path = [data_path])

In [19]:
#from train import clean_data

# Use the clean_data function to clean your data.
#x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [20]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=15,
    task= 'classification',
    primary_metric='AUC_weighted',
    training_data=automl_data,
    compute_target=compute_target,
    label_column_name= 'y',
    n_cross_validations= 3)

In [21]:
# Submit your automl run
automl_experiment = Experiment(ws, 'automl_exp')
automl_run = automl_experiment.submit(automl_config, show_output=True)

Running on remote or ADB.
Running on remote compute: comp-cluster1
Parent Run ID: AutoML_03e629e5-3e86-465c-aee9-f7c6c83735cd

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input d

In [22]:
from azureml.widgets import RunDetails
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

### Get the best model and the values of the performance metrics

In [24]:
# Retreive thw best model
best_run, fitted_model = automl_run.get_output()
print(best_run)
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Run(Experiment: automl_exp,
Id: AutoML_03e629e5-3e86-465c-aee9-f7c6c83735cd_10,
Type: azureml.scriptrun,
Status: Completed)
accuracy 0.9157813985272036
balanced_accuracy 0.7570403516305899
f1_score_micro 0.9157813985272036
recall_score_weighted 0.9157813985272036
log_loss 0.1789443912594687
AUC_micro 0.9810658824133753
f1_score_macro 0.7739150377184937
f1_score_weighted 0.9128550239091021
recall_score_macro 0.7570403516305899
matthews_correlation 0.5508731684994235
weighted_accuracy 0.9552602467734074
precision_score_macro 0.7955272190498421
precision_score_weighted 0.9111497633175466
AUC_weighted 0.9488766254010765
norm_macro_recall 0.5140807032611797
AUC_macro 0.9488766362029327
average_precision_score_macro 0.8271635433162032
recall_score_micro 0.9157813985272036
precision_score_micro 0.9157813985272036
average_precision_score_micro 0.9818432424307814
average_precision_score_weighted 0.956125486801013
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_03e629e5-3e86-465c-aee

In [27]:
# Retrieve and save your best automl model.
best_run.register_model(model_path='outputs/model.pkl', model_name='automl_model', tags={'Training context':'Auto ML'}, 
                        properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']})

Model(workspace=Workspace.create(name='Udacity-project', subscription_id='de47103e-2da6-4f5e-88fc-d18b27fd249b', resource_group='aml-quickstarts-112571'), name=automl_model, id=automl_model:1, version=1, tags={'Training context': 'Auto ML'}, properties={'AUC': '0.9488766254010765', 'Accuracy': '0.9157813985272036'})

## Shut down and delete the compute cluster once all the experiments are complete

In [ ]:
compute_target.delete()